In [2]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

In [3]:
# Now we need to lookup the filename for xemnas's mdlx object (where the AI file is kept)
# This method will return information on all the different Xemnas's, so make sure to locate the correct one
lib.get_object(name="luxord")

[{'ucm': '09C8', 'mdlx': 'B_EX150_LV99', 'name': 'Data Luxord'},
 {'ucm': '05F8', 'mdlx': 'B_EX150', 'name': 'Luxord'},
 {'ucm': '05FF', 'mdlx': 'B_EX210', 'name': "Luxord's Card"}]

In [29]:
# Now we extract the xemnas BAR into a folder in our workspace
MDLX = "B_EX150.mdlx"
BAR = os.path.join(lib.gitpath, "KH2", "obj", MDLX)
assert os.path.isfile(BAR)
lib.editengine.bar_extract(BAR, os.path.join(os.getcwd(), "workspace",MDLX))

In [37]:
syscalls = ''
syscalls = {'func_system_set_game_speed': {'tableidx': 1, 'funcidx': 27}, 'trap_mission_failed': {'tableidx': 4,'funcidx': 11}}
commands = {
    'ret': [0x89, 0x0, 0x0, 0x0],
    'push.v': [0x0, 0x0, 0x0, 0x0, 0x0, 0x0],
    'syscall': [0x0a, 0x0, 0x0, 0x0]
}
def get_mod_bytestr(code, debug=False):
    bta = list()
    commandlist = code.split("\n")
    for command in commandlist:
        values = command.split(" ")
        if values[0] == 'ret':
            bta += list(commands['ret'])
        elif values[0] == 'push.v':
            temp = list(commands['push.v'])
            if not len(values) == 2:
                raise Exception("Incorrect number of values for {}".format(command))
            val = int(values[1])
            if val > 255:
                raise Exception("Can't push number more than 255: {}".format(values[1]))
            temp[2] = val
            bta += temp
        elif values[0] == 'syscall':
            if not len(values) == 2:
                raise Exception("Incorrect number of values for {}".format(command))
            call = values[1]
            if not call in syscalls:
                raise Exception("Syscall name not found: {}".format(call))
            sc = syscalls[call]
            ti = sc["tableidx"]
            temp = list(commands['syscall'])
            temp[0] = temp[0] + ((ti % 4)*(16*4))
            temp[1] = ti // 4
            temp[2] = sc["funcidx"]
            bta += temp
        else:
            raise Exception("Command not recognized: {}".format(values[0]))
    if debug:
        print(bta)
    return bta


In [23]:
0x0a + ((1 % 4)*(16*4))

74

In [33]:
import shutil
shutil.copy(base_file, mod_file)

'workspace\\B_EX150.mdlx\\b_ex.bdx.bin'

In [43]:
def write_code(syscall, var1=None, var2=None, var3=None, var4=None, var5=None, debug=False):
    code = ''
    if var1:
        code += 'push.v {}\n'.format(var1)
    if var2:
        code += 'push.v {}\n'.format(var2)
    if var3:
        code += 'push.v {}\n'.format(var3)
    if var4:
        code += 'push.v {}\n'.format(var4)
    if var5:
        code += 'push.v {}\n'.format(var5)
    code += 'syscall {}\n'.format(syscall)
    code += 'ret'
    if debug:
        print(code)
    return code.strip()

In [48]:
import shutil
base_file = os.path.join("workspace", "B_EX150.mdlx", "b_ex.bdx")
mod_file = os.path.join("workspace", "B_EX150.mdlx", "b_ex.bdx.bin")
assert os.path.isfile(base_file)
shutil.copy(base_file, mod_file)
assert os.path.isfile(mod_file)


debug = False
codedebug = True
OFFSET_FOR_INJECTION = 0x1e5a

var1 = 15
var2 = None
var3 = None
var4 = None
var5 = None
syscall = 'func_system_set_game_speed'

mod_str = get_mod_bytestr(write_code(syscall, var1=var1, var2=var2, var3=var3, var4=var4, var5=var5, debug=codedebug), debug=debug)

mod_bytes = bytearray(open(mod_file, "rb").read())

for b in range(len(mod_str)):
    bt = mod_str[b]
    mod_bytes[OFFSET_FOR_INJECTION + b] = bt

open(mod_file, "wb").write(mod_bytes)

dumpname = "luxinpete"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))

starting_offset = int(lib.parsingengine.locate_file(base_file),16)

lib.cheatengine.pnach = []
# This code will diff the two versions of the ai file and create codes to make those same changes via RAM
codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
for code in codes:
    print(code)
print(codes)
# The memdump skips the first section of memory addresses so the code offsets need to be fixed
codes = [code[1].replace("0","2") + code[1:] for code in codes]
codes
# codes = []#DEBUG

# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

push.v 15
syscall func_system_set_game_speed
ret
010f20b4 00000089
010f20b8 0000000f
010f20bc 001b004a
010f20c0 00000089
['010f20b4 00000089', '010f20b8 0000000f', '010f20bc 001b004a', '010f20c0 00000089']
patch=1,EE,110F20B4,extended,00000089

patch=1,EE,110F20B8,extended,0000000F

patch=1,EE,110F20BC,extended,001B004A

patch=1,EE,110F20C0,extended,00000089



In [ ]:
+0x1000

In [ ]:
import time, os
while 1:
    time.sleep(0.1)
    if os.path.isfile(mod_file):
        lib.cheatengine.pnach = []
        codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
        codes = [code[1].replace("1","2") + code[1:] for code in codes]
        lib.cheatengine.apply_ram_code(codes)
        lib.cheatengine.write_pnach(debug=False)
        print(time.time())
        os.remove(mod_file)

In [ ]:
# Code to Roxas With Xemnas, and create the pnach file
lib.cheatengine.apply_ram_code(["11C4F008 00000646"])
lib.cheatengine.write_pnach()

In [ ]:
0x10f9ad0-0x254c

In [ ]:
starting_offset

In [ ]:
#Rikus adjustments
# 0x10f7584 WHERE RIKU AI FILE STARTS

#//Remove Riku's HP Barrier 1
patch=1,EE,01C6BDB8,extended,00000000
//Remove Riku's HP Barrier 2
patch=1,EE,01C6C020,extended,00000000
//Remove Riku's HP Barrier 3
patch=1,EE,01C6C288,extended,00000000
""".upper()


In [ ]:
# This method will look through the memdump to try and find where the ai file is located. 
# If you already know the offset where the file starts, this is uneccessary 
starting_offset = int(lib.parsingengine.locate_file(base_file),16)
hex(starting_offset)

In [ ]:
# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

In [ ]:
## OK I need to first verify it between the 1st and 2nd enemies since that is where its easier to hand do this
STARTING_OFFSET = 0x1d119ac
FILE_HEADER = 0x8
HP_OFF_INDEX = 0x4
INDEX = 106
LENGTH_PER_I = 0x5C

calculated_offset = STARTING_OFFSET + FILE_HEADER + (LENGTH_PER_I*INDEX) + HP_OFF_INDEX
calculated_offset = hex(calculated_offset)
RIGHT_OFFSET = '0x01d13fd0'
assert calculated_offset==RIGHT_OFFSET, hex(int(calculated_offset,16)-int(RIGHT_OFFSET,16))

In [ ]:
base_file = os.path.join("workspace", "battle", "enmp.list.base")
inds = {}
base = bytearray(open(base_file, "rb").read())
i=0
while i < 229:
    si = LENGTH_PER_I*i+8
    o = {
        "index": i,
        "id": base[si],
        "hp": base[si+4]+255*base[si+5]
    }
    if base[si+6] != 0 or base[si+7] != 0:
        o["extra_hp_bytes"] = "{}{}".format(hex(base[si+7])[2:].zfill(2),hex(base[si+6])[2:].zfill(2))
    #print(list(base[LENGTH_PER_I*i+8:LENGTH_PER_I*i+8+16]))
    inds[o["id"]] = o
    i+=1

In [ ]:
import json
pth = os.path.join("..","..","..","..","kh2-spawn-replacer","enemies.json")
d = json.load(open(pth))
for k in d:
    if "enmp" in d[k]:
        oj = inds[d[k]["enmp"]]
        d[k]["enmp"] = oj["index"]
        d[k]["hp"] = oj["hp"]
        if "extra_hp_bytes" in oj:
            d[k]["hp_extra_bytes"] = oj["extra_hp_bytes"]
json.dump(d, open(pth,"w"), indent=2)

In [ ]:
d[k]

In [ ]:
# For each memdump (should be named after the msn)
   # find the msn starting point (list file)
   # get the associated mdlx file, 
      # extract it, get the ai offset and the mdlx offset
   # create a dict with this info
# Start with Twilight Thorn, Demyx, Xemnas, Data Demyx, Data Xemnas
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
information = {}
parsing_path = os.path.join(os.getcwd(),"workspace", "parsing")
if not os.path.isdir(parsing_path):
    os.mkdir(parsing_path)
pth = os.path.join("..","..","..","..","kh2-spawn-replacer","enemies.json")
enemies_list = json.load(open(pth))
dumpdir = 'C:\\Users\\12sam\\Desktop\\xeemo'
bad_dumps = []
dumps = list(bad_dumps) or os.listdir(dumpdir)
for dumpfn in dumps:
    try:
        print(dumpfn)
        msn_name = dumpfn.split("(")[0].strip()
        doMSN = msn_name != 'NONE'
        enemy_name = dumpfn.split("(")[1].split(")")[0]
        mdlx_name = enemies_list[list(filter(lambda k: enemies_list[k]['name'] == enemy_name, enemies_list))[0]]['model']
        if doMSN:
            msn_path = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "msn", "jp", msn_name+"."+"bar")
            assert os.path.isfile(msn_path)
            msn_out_path = os.path.join(parsing_path,msn_name)
        enemy_path = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "obj", mdlx_name+".mdlx")
        enemy_out_path = os.path.join(parsing_path,enemy_name)
        if doMSN:
            lib.editengine.bar_extract(msn_path, msn_out_path)
        lib.editengine.bar_extract(enemy_path, enemy_out_path)
        if doMSN:
            msn_list_path = os.path.join(msn_out_path, [p for p in os.listdir(msn_out_path) if p.endswith('.list')][0])
        mdlx_ai_path = os.path.join(enemy_out_path, [p for p in os.listdir(enemy_out_path) if p.endswith('.ai')][0])
        subdata = lib.parsingengine.load_memdump(os.path.join(dumpdir, dumpfn))

        if doMSN:
            msn_offset = lib.parsingengine.locate_file(msn_list_path, single_result=True)
        ai_offset_dump = lib.parsingengine.locate_file(mdlx_ai_path, single_result=True)

        subdata_mdlx = lib.parsingengine.load_memdump(enemy_path)
        ai_offset_mdlx = lib.parsingengine.locate_file(mdlx_ai_path,single_result=True)

        data = {
            "mdlx_offset": hex(int(ai_offset_dump,16) - int(ai_offset_mdlx,16)),
            "ai_start_offset": ai_offset_mdlx
        }
        if doMSN:
            data["msn_offset"] = msn_offset

        information[enemy_name] = data
    except Exception as exc:
        print(" failed: {}".format(exc))
        
information

In [ ]:
sub = bytearray(b'\x02\x00\xdc\x00`\x00\xd0&\x05\x00\x7f;\x00\x00\x02\x00\x00\x00\x00\x005\x00\x00\x00\x00\x00\x00\x00\x02\x00\x01\x00\x02\x006\x02\x04\x00\x04\x00x\x00\x00\x00x\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x05\x00\x04\x00\x00\x00\x00\x00\x04\x00\x00@\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00')
lib.parsingengine.search_substr(sub)

In [ ]:
location_pth = os.path.join("..","..","..","..","kh2-spawn-replacer","locations.json")
locations = json.load(open(location_pth))
for k in information:
    data = information[k]
    if "ai_start_offset" in data:
        enemies_list[k]["ai_start_offset"] = data["ai_start_offset"]
    if "msn_offset" in data or "mdlx_offset" in data:
        for loc in locations:
            if k in [e["name"] for e in loc["enemies"]]:
                if "msn_offset" in data:
                    loc["msn_offset"] = data["msn_offset"]
                if "mdlx_offset" in data:
                    for e in loc["enemies"]:
                        if k == e["name"]:
                            e["mdlx_offset"] = data["mdlx_offset"]
json.dump(enemies_list, open(pth,"w"), indent=2)
json.dump(locations, open(location_pth,"w"), indent=2)

In [ ]:
# The double battles and armor xemnases are special and need manual handling

In [ ]:
os.listdir(out)

In [ ]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json, shutil
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

BAR = os.path.join(lib.gitpath, "KH2", "obj", "B_CA010.mdlx")

out = os.path.join(os.getcwd(), "workspace", "barbossa")

lib.editengine.bar_extract(BAR, out)

ai_fn = os.path.join(out,"b_ca.bdx")
base_fn = ai_fn+".base"

shutil.copy(ai_fn, base_fn)

lib.parsingengine.set_file_bytes(
    fn=ai_fn, 
    proportions=[0], 
    byte_to_set=0x0)

In [ ]:
dumpname = "barbinpete"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))
import time
starting_offset = int(lib.parsingengine.locate_file(base_fn), 16)
while 1:
    time.sleep(0.05)
    lib.cheatengine.pnach = []

    codes = lib.parsingengine.generate_codes(str(starting_offset), base_fn, ai_fn)

    codes = [code[1].replace("1","2") + code[1:] for code in codes]

    lib.cheatengine.apply_ram_code(codes)
    lib.cheatengine.write_pnach(debug=False)

In [ ]:
base_file = os.path.join("workspace", "pete", "n_wi.bdx")
mod_file = os.path.join("workspace", "xemnas", "n_wi.bdx.bin")
assert os.path.isfile(base_file)
assert os.path.isfile(mod_file)

# go to the roxas fight, and create a memdump using emuhaste
#  it is recommended to make a savestate in the cutscene before the fight loads, for ease of testing
# Place the memdump in the workspace folder, using whatever name you choose
dumpname = "peteinrox"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))
#SUBDATA = lib.parsingengine.load_memdump(base_file) #IF THINGS ARENT WORKING LOOK HERE SILLY

starting_offset = int(lib.parsingengine.locate_file(base_file),16)

lib.cheatengine.pnach = []
# This code will diff the two versions of the ai file and create codes to make those same changes via RAM
codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
for code in codes:
    print(code)
print(codes)
# The memdump skips the first section of memory addresses so the code offsets need to be fixed
codes = [code[1].replace("0","2") + code[1:] for code in codes]
codes
# codes = []#DEBUG

# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

In [11]:
lib.patch_game(fromgit=True)

['-patchmaker', '-output', 'patch.kh2patch', '-author', 'kh2lib', '-version', '1', '-skipchangelog', '-skipcredits']
(b'KH2FM_Toolkit\nVersion PRE-3.0\nDEVELOPER RELEASE\n\nProgrammed by GovanifY\nhttp://www.govanify.com\nhttp://www.twitter.com/GovanifY\nSoftware under GPL 2 license, for more info, use the command -license\n\nThis tool is able to create patches for the software KH2FM_Toolkit.\nIt can add files using the internal compression of the game \nKingdom Hearts 2(Final Mix), relink files to others, recreate\nthe iso without size limits and without corruption.\nThis patch system is the best ever made for this game atm.\n\nPress enter to run using the file: patch.kh2patchOther information (leave blank to continue): Filenames may be formatted as text (msg/jp/lk.bar) or hash (0x030b45da).\r\n\nEnter filename:   Using "msn/jp/MU02_MS103B.bar" for 95FBC2CA\r\nRelink to this filename(ex: 000al.idx) [Blank for none]: Compress this file? [Y/n] Parent compressed file [Leave blank for KH2

In [ ]:
rp

In [ ]:
replace = not replace

In [ ]:
replace

In [ ]:
readfad